In [1]:
import difflib
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.interfaces.rolodex_interface import rolodex_load
from libs.reference.constants import COMPONENT_TYPE_MAP, PAPER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

In [31]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
data = rolodex_load(
    "inspection", 
    links=[
        "component", 
        "unit", 
        "site", 
        "organization"
    ], 
    filters=[
        {
            "type": "component", 
            "key": "portal.component_type", 
            "value": "2"
        }
    ], 
    ids=[], 
    tags={
    }, 
    limit=100000
)

In [3]:
tank_shell_inspections_df = pd.json_normalize(data)

In [4]:
tank_shell_inspections_df.shape

(1981, 271)

In [5]:
tank_shell_inspections_df.head()

,entry.id,entry.type,entry.data.date,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,...,entry.data.total_calibration,entry.data.total_rc_duration,entry.data.num_status_updates,entry.data.total_run_duration,entry.data.total_shift_break_duration,entry.data.description,links.component.data.description,entry.data.shipview.inspection_display.horizontal_surface,entry.tags.unit-type,entry.tags.component-type
0,20170424-0e1e98,inspection,2017-04-24,19.0,2017-04-24,0.0,None,20170424-0e1e98,,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20170424-503cda,inspection,2017-04-24,21.0,2017-04-24,0.0,None,20170424-503cda,,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20170424-90eedb,inspection,2017-04-24,18.0,2017-04-24,0.0,None,20170424-90eedb,,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20170728-5ade01,inspection,2017-07-28,1606.0,2017-07-28,0.0,None,20170728-5ade01,,394.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20170424-dfad9b,inspection,2017-04-24,20.0,2017-04-24,0.0,None,20170424-dfad9b,,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
tank_shell_inspections_df['entry.data.portal.date'].min()

'2017-04-24'

In [7]:
tank_shell_inspections_df['entry.data.portal.date'].max()

'2023-07-18'

In [8]:
completed_tank_shell_inspections_df = tank_shell_inspections_df[tank_shell_inspections_df['entry.data.portal.status']=='completed']

In [9]:
completed_tank_shell_inspections_df['entry.data.portal.date'].max()

'2023-06-23'

In [29]:
completed_tank_shell_inspections_df['links.organization.data.name'].astype(str).str.lower()

0                              aes
1                              aes
2                              aes
3                  georgia pacific
4                              aes
                   ...            
1959      par pacific holdings inc
1960                      cardinal
1962    alliant energy (corporate)
1964      par pacific holdings inc
1967                           nan
Name: links.organization.data.name, Length: 1852, dtype: object

In [32]:
completed_tank_shell_inspections_df.groupby(['links.organization.data.name'])['entry.id'].nunique().to_frame()

,entry.id
links.organization.data.name,
ADNOC - Abu Dhabi National Oil Company,1
ADNOC Gas Processing,1
AEP,15
AES,54
AK Steel,1
APS,9
Afton Chemical,10
Alliant Energy (Corporate),2
American Crystal Sugar,28


In [35]:
big_west_df = completed_tank_shell_inspections_df[completed_tank_shell_inspections_df['links.organization.data.name']=='Big West Oil']

In [36]:
big_west_df.head()

,entry.id,entry.type,entry.data.date,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.data.status,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,entry.tags.inspection-type,links.component.id,links.component.type,links.component.data.name,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.name,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.name,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.address.city,links.site.data.address.region,links.site.data.address.country,links.site.data.address.postal_code,links.site.data.address.address_line_1,links.site.data.address.address_line_2,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.plant_type,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.sf-account,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.name,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id

In [45]:
big_west_df['links.component.data.portal.shell_diameter'].min()

0.0

In [46]:
big_west_df['links.component.data.portal.shell_diameter'].max()

80.0

In [47]:
unique_big_west_component_slugs = list(set(big_west_df['links.component.data.portal.slug']))

In [48]:
unique_big_west_component_slugs

['shell-7200ff',
 'roof-f4f612',
 'roof-3aa07a',
 'tank-shell-70b356',
 'shell-6c6d43',
 'shell-e4cf38',
 'shell-47ec70',
 'shell-a134aa',
 'shell-e7b8dc',
 'shell-2bde02',
 'floor-84b20c',
 'shell-156330',
 'roof-f4d1f0',
 'shell-da2430',
 'shell-9a0338',
 'shell-b085f6']

In [20]:
list(completed_tank_shell_inspections_df.columns)

['entry.id',
 'entry.type',
 'entry.data.date',
 'entry.data.portal.id',
 'entry.data.portal.date',
 'entry.data.portal.demo',
 'entry.data.portal.name',
 'entry.data.portal.slug',
 'entry.data.portal.notes',
 'entry.data.portal.job_id',
 'entry.data.portal.status',
 'entry.data.portal.read_only',
 'entry.data.portal.show_excel',
 'entry.data.portal.sort_order',
 'entry.data.portal.data_import',
 'entry.data.portal.external_id',
 'entry.data.portal.zero_offset',
 'entry.data.portal.component_id',
 'entry.data.portal.inspection_type',
 'entry.data.portal.ndt_method_type',
 'entry.data.portal.target_release_time',
 'entry.data.portal.sales_robotic_service_id',
 'entry.data.status',
 'entry.tags.job',
 'entry.tags.site',
 'entry.tags.unit',
 'entry.tags.component',
 'entry.tags.inspection',
 'entry.tags.organization',
 'entry.tags.inspection-type',
 'links.component.id',
 'links.component.type',
 'links.component.data.name',
 'links.component.data.portal.id',
 'links.component.data.portal

In [10]:
sliced_inspection_df = completed_tank_shell_inspections_df.loc[:, ('entry.data.portal.slug', 'links.component.data.portal.wall_nominal_thickness')].reset_index()

In [11]:
sliced_inspection_df.rename(
    columns={
        'entry.data.portal.slug': 'slug',	
        'links.component.data.portal.wall_nominal_thickness': 'nominal',
    }, 
    inplace=True
)
sliced_inspection_df.drop(columns={'index'}, inplace=True)

In [12]:
sliced_inspection_df.head()

,slug,nominal
0,20170424-0e1e98,0.325
1,20170424-503cda,0.375
2,20170424-90eedb,0.325
3,20170728-5ade01,0.500
4,20170424-dfad9b,0.325


In [13]:
sliced_inspection_df.nominal.describe()

count    1590.000000
mean        0.799960
std        12.956031
min         0.000000
25%         0.250000
50%         0.375000
75%         0.500000
max       516.000000
Name: nominal, dtype: float64

In [14]:
clean_nominals_df = sliced_inspection_df[
    ~sliced_inspection_df['nominal'].isna() &
    (sliced_inspection_df['nominal']!=0) &
    (sliced_inspection_df['nominal'] < 1) &
    (sliced_inspection_df['nominal'] > .01)
]

In [15]:
clean_nominals_df.nominal.describe()

count    1493.000000
mean        0.390356
std         0.166638
min         0.078000
25%         0.250000
50%         0.375000
75%         0.500000
max         0.969000
Name: nominal, dtype: float64

In [16]:
clean_nominals_df.reset_index(inplace=True)

In [17]:
histogram_df, hist_error_slug_list = get_thickness_histogram(clean_nominals_df)

0/1493
10/1493
20/1493
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
30/1493
40/1493
50/1493
60/1493


KeyboardInterrupt: 

In [ ]:
histogram_df.shape

In [ ]:
len(hist_error_slug_list)

In [ ]:
histogram_df.head()

In [ ]:
comb_df = histogram_df.merge(tank_shell_inspections_df, how='left', left_on='slug', right_on='entry.data.portal.slug')

In [ ]:
comb_df.head()

In [ ]:
list(comb_df.columns)

In [ ]:
trimmed_comb_df = comb_df.loc[:,(
     'slug',
     'nominal',
     'min_t',
     'max_t',
     'tubes_inspected',
     'bins_collected',
     'crits_per10k',
     'Bins w 10% Loss',
     'Bins w 20% Loss',
     'Bins w 30% Loss',
     'Bins w 40% Loss',
     'Bins w 50% Loss',
     'Bins w 60% Loss',
     'Bins w 70% Loss',
     'Bins w 80% Loss',
     'Bins w 90% Loss',
     'entry.data.portal.date',
     'entry.data.portal.demo',
     'entry.tags.job',
     'entry.tags.site',
     'entry.tags.unit',
     'entry.tags.component',
     'entry.tags.organization',
     'links.component.data.portal.shell_diameter',
     'links.unit.data.portal.name',
     'links.site.data.portal.name',
     'links.organization.data.portal.name',
)]

In [ ]:
trimmed_comb_df.head()

In [ ]:
trimmed_comb_df.sort_values(by=['crits_per10k'], ascending=False)

In [ ]:
set(trimmed_comb_df['entry.data.portal.demo'])

In [ ]:
set(trimmed_comb_df['links.unit.data.portal.name'])

In [ ]:
tanks_df = trimmed_comb_df[trimmed_comb_df['links.unit.data.portal.name'].apply(lambda x: 'tank' in x.lower())]

In [ ]:
tanks_df.head()

In [ ]:
set(tanks_df['links.unit.data.portal.name'])

In [ ]:
tanks_df.shape

In [ ]:
set(tanks_df['links.component.data.portal.shell_diameter'])

In [ ]:
tanks_df.to_csv('../data/outputs/paper_mill_tanks.csv')